<a href="https://colab.research.google.com/github/HaedaraHamed/Second-Network-Programming-Homework/blob/main/voice_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
import os
import numpy as np
import IPython.display as ipd
from sklearn.preprocessing import LabelEncoder

!apt-get -qq install -y utils && pip install np_utils
!pip install np_utils
!pip install keras


from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D

from keras.models import Model

from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
!sudo apt-get install libportaudio2
!pip install sounddevice
import soundfile as sf
import sounddevice as sd
import np_utils
from keras.utils import to_categorical
import shutil


E: Unable to locate package utils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 698.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=123e72e6c5c44e246ad312d213a9f262d01c0976571c9c6214588c3c088b65d8
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 1s (130 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is instal

In [ ]:
# in this way ...during we are using collab we are saving (training file for neural network)  on drive
#to continue training from the last point we reachedin training
from google.colab import drive

drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# making file and named it "data" to save dataset in it then extract dataset in data file

# Create the "data" folder inside the parent directory
data_dir = os.path.join("data")
os.mkdir(data_dir)

# Verify the current working directory
current_directory = os.getcwd()



In [ ]:
# Change the current working directory to "/content/data"
os.chdir("/content/data")
!pwd

/content/data


In [ ]:
#import training files from the "tensorflow"
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz


--2024-06-12 07:47:21--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.193.207, 142.250.152.207, 173.194.194.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.193.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G   177MB/s    in 13s     

2024-06-12 07:47:34 (113 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]



In [ ]:
#extract data in  "/content/data"
!tar -xvf '/content/data/speech_commands_v0.01.tar.gz'

Streaming output truncated to the last 5000 lines.
./up/6f9088d7_nohash_0.wav
./up/6f342826_nohash_0.wav
./up/e0a7c5a0_nohash_0.wav
./up/4d4e17f5_nohash_1.wav
./up/b0f24c9b_nohash_0.wav
./up/735845ab_nohash_2.wav
./up/53d5b86f_nohash_0.wav
./up/1a5b9ca4_nohash_1.wav
./up/23abe1c9_nohash_2.wav
./up/bdee441c_nohash_1.wav
./up/a1cff772_nohash_1.wav
./up/1ecfb537_nohash_3.wav
./up/37fc5d97_nohash_3.wav
./up/bd8412df_nohash_1.wav
./up/e53139ad_nohash_1.wav
./up/10ace7eb_nohash_3.wav
./up/30065f33_nohash_0.wav
./up/eefd26f3_nohash_0.wav
./up/c9b653a0_nohash_2.wav
./up/02746d24_nohash_0.wav
./up/e1469561_nohash_0.wav
./up/4bba14ce_nohash_0.wav
./up/b5d1e505_nohash_1.wav
./up/531a5b8a_nohash_1.wav
./up/0135f3f2_nohash_0.wav
./up/dbb40d24_nohash_4.wav
./up/e9287461_nohash_1.wav
./up/71e6ab20_nohash_0.wav
./up/ead2934a_nohash_1.wav
./up/f9af0887_nohash_0.wav
./up/ff63ab0b_nohash_0.wav
./up/f3d06008_nohash_0.wav
./up/918a2473_nohash_4.wav
./up/e54a0f16_nohash_0.wav
./up/cb8f8307_nohash_1.wav
./up

In [ ]:
#delated unused file


file_path = "/content/data/LICENSE"
os.remove(file_path)
file_path = "/content/data/README.md"
os.remove(file_path)
file_path = "/content/data/speech_commands_v0.01.tar.gz"
os.remove(file_path)
file_path = "/content/data/testing_list.txt"
os.remove(file_path)
file_path = "/content/data/validation_list.txt"
os.remove(file_path)

folder_path = "/content/data/_background_noise_"
shutil.rmtree(folder_path)




In [ ]:
train_audio_path = '/content/data'
# readind all files in the path below
labels = os.listdir(train_audio_path)

all_wave = []

all_label = []

for label in labels:
    print(label)
    # give what the file have in inside (all files such as (bird has sounds >> enter to it and show me this sound ))

    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    #after give me all files.wav
    #librosa reading soundfile & return samples & sample rate

    for wav in waves:
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, orig_sr=sample_rate, target_sr=8000)
        #samples = librosa.resample(samples, sample_rate, 8000)
        if len(samples)== 8000 :
             all_wave.append(samples)
             all_label.append(label)
# in training we give him input and output

six
five
three
go
right
house
down
off
on
wow
sheila
zero
left
dog
tree
stop
cat
seven
nine
marvin
happy
eight
bird
yes
up
no
one
four
bed
two


In [ ]:
print(all_label)

['six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six', 'six'

In [ ]:
# from sk learn make data (samples) ready  for inter to neural  network
le = LabelEncoder()
y=le.fit_transform(all_label)#numberd samples
classes= list(le.classes_)


In [ ]:
print(classes)

['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [ ]:
import np_utils
from keras.utils import to_categorical
y=to_categorical(y, num_classes=len(labels))

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
all_wave = np.array(all_wave).reshape(-1,8000,1)
#because list cant inter to neural  network we make all_wave an array  and reshape it


In [ ]:
all_wave

array([[[-1.7295752e-04],
        [-1.7285880e-04],
        [-4.6212145e-04],
        ...,
        [-4.1556079e-04],
        [ 3.7281885e-04],
        [ 6.7376794e-04]],

       [[ 2.6436715e-04],
        [ 8.6898886e-04],
        [ 1.0412681e-03],
        ...,
        [-5.3666468e-04],
        [-1.2616417e-03],
        [-1.5709300e-03]],

       [[ 1.3069862e-03],
        [ 1.5917599e-03],
        [-1.1239555e-03],
        ...,
        [ 2.3027195e-04],
        [ 5.1642372e-04],
        [ 1.0342512e-04]],

       ...,

       [[ 5.3171098e-06],
        [ 5.6793739e-05],
        [ 3.9552182e-05],
        ...,
        [-3.3087839e-05],
        [-6.0474929e-05],
        [-7.1938390e-05]],

       [[-4.7081630e-06],
        [ 1.5960669e-04],
        [ 9.3091978e-05],
        ...,
        [-5.8838661e-05],
        [ 1.1615795e-04],
        [-5.2921296e-06]],

       [[ 2.0950101e-04],
        [ 2.8002001e-03],
        [ 5.9715118e-03],
        ...,
        [-9.1740415e-03],
        [-7.134

In [ ]:
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)
#sk learn after train go to validation to evalution


In [ ]:
model=load_model('/content/drive/MyDrive/best_model.hdf5')


In [ ]:
K.clear_session()

inputs = Input(shape=(8000,1))

#First Conv1D layer
conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Second Conv1D layer
conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Third Conv1D layer
conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Fourth Conv1D layer
conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Flatten layer
conv = Flatten()(conv)

#Dense Layer 1
conv = Dense(256, activation='relu')(conv)
conv = Dropout(0.3)(conv)

#Dense Layer 2
conv = Dense(128, activation='relu')(conv)
conv = Dropout(0.3)(conv)

outputs = Dense(len(labels), activation='softmax')(conv)

model = Model(inputs, outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 7988, 8)           112       
                                                                 
 max_pooling1d (MaxPooling1  (None, 2662, 8)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 2662, 8)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2652, 16)          1424      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 884, 16)           0         
 g1D)                                                        

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001)
mc = ModelCheckpoint('/content/best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')


In [ ]:
history=model.fit(x_tr, y_tr ,epochs=100, callbacks=[es,mc], batch_size=32, validation_data=(x_val,y_val))


Epoch 1/100
1457/1457 [==============================] - ETA: 0s - loss: 2.9223 - accuracy: 0.1316

1457/1457 [==============================] - 22s 10ms/step - loss: 2.9223 - accuracy: 0.1316 - val_loss: 2.4325 - val_accuracy: 0.2753
Epoch 2/100
1451/1457 [============================>.] - ETA: 0s - loss: 1.9561 - accuracy: 0.3856

1457/1457 [==============================] - 13s 9ms/step - loss: 1.9544 - accuracy: 0.3861 - val_loss: 1.5638 - val_accuracy: 0.5224
Epoch 3/100
1454/1457 [============================>.] - ETA: 0s - loss: 1.5113 - accuracy: 0.5282

1457/1457 [==============================] - 17s 11ms/step - loss: 1.5109 - accuracy: 0.5283 - val_loss: 1.2760 - val_accuracy: 0.6182
Epoch 4/100
1455/1457 [============================>.] - ETA: 0s - loss: 1.3166 - accuracy: 0.5931

1457/1457 [==============================] - 15s 10ms/step - loss: 1.3168 - accuracy: 0.5930 - val_loss: 1.0431 - val_accuracy: 0.6975
Epoch 5/100
1457/1457 [==============================] - ETA: 0s - loss: 1.1933 - accuracy: 0.6349

...

1457/1457 [==============================] - 13s 9ms/step - loss: 0.6333 - accuracy: 0.8062 - val_loss: 0.5958 - val_accuracy: 0.8282
Epoch 30/100
1454/1457 [============================>.] - ETA: 0s - loss: 0.6252 - accuracy: 0.8063

1457/1457 [==============================] - 13s 9ms/step - loss: 0.6253 - accuracy: 0.8063 - val_loss: 0.5980 - val_accuracy: 0.8239
Epoch 31/100
1454/1457 [============================>.] - ETA: 0s - loss: 0.6256 - accuracy: 0.8065

1457/1457 [==============================] - 13s 9ms/step - loss: 0.6257 - accuracy: 0.8065 - val_loss: 0.6173 - val_accuracy: 0.8188
Epoch 32/100
1454/1457 [============================>.] - ETA: 0s - loss: 0.6200 - accuracy: 0.8094

1457/1457 [==============================] - 13s 9ms/step - loss: 0.6200 - accuracy: 0.8094 - val_loss: 0.6155 - val_accuracy: 0.8222
Epoch 33/100
1455/1457 [============================>.] - ETA: 0s - loss: 0.6129 - accuracy: 0.8117

1457/1457 [==============================] - 14s 10ms/step - loss: 0.6132 - accuracy: 0.8116 - val_loss: 0.6142 - val_accuracy: 0.8207
Epoch 33: early stopping


In [ ]:
model.save(filepath="/content/drive/MyDrive/best_model.hdf5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def predict(audio):
    prob=model.predict(audio.reshape(1,8000,1))
    index=np.argmax(prob[0])
    return classes[index]


In [ ]:

samples, sample_rate = librosa.load("/content/data/up/00b01445_nohash_0.wav", sr = 16000)
samples = librosa.resample(samples, orig_sr=sample_rate, target_sr=8000)
ipd.Audio(samples, rate=8000)


In [ ]:
print("Text:",predict(samples))

1/1 [==============================] - 0s 368ms/step
Text: up


In [ ]:
from IPython.display import HTML

AUDIO_HTML = """
<script>
async function recordAudio() {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const recorder = new MediaRecorder(stream);
    const chunks = [];

    recorder.ondataavailable = (e) => chunks.push(e.data);
    recorder.onstop = (e) => {
        const blob = new Blob(chunks, { type: 'audio/wav' });
        const url = URL.createObjectURL(blob);
        const audio = new Audio(url);
        audio.controls = true;
        document.body.appendChild(audio);
    };

    recorder.start();
    await sleep(1000);  // Record for 5 seconds (adjust as needed)
    recorder.stop();
}

function sleep(time) {
    return new Promise((resolve) => setTimeout(resolve, time));
}
</script>
<button onclick="recordAudio()">Record Audio</button>
"""

display(HTML(AUDIO_HTML))


In [ ]:
def predict(audio):
    # Zero-pad or trim the audio to 8000 samples
    if len(audio) < 8000:
        audio = np.pad(audio, (0, 8000 - len(audio)), mode='constant')
    elif len(audio) > 8000:
        audio = audio[:8000]

    prob = model.predict(audio.reshape(1, 8000, 1))
    index = np.argmax(prob[0])
    return classes[index]


In [ ]:
samples, sample_rate = librosa.load("/content/data/zero/012c8314_nohash_0.wav", sr = 16000)
samples = librosa.resample(samples, orig_sr=sample_rate, target_sr=8000)
ipd.Audio(samples, rate=8000)

In [ ]:
#converting voice commands to text
print("Text:",predict(samples))

1/1 [==============================] - 0s 36ms/step
Text: up
